# Metaclass vs class decorator

As we have seen, class decorators can achieve a lot of the metaprogramming goals we might have.

But there is one area where they fall short of metaclasses - inheritance.

Metaclasses are carried through inheritance, whereas decorators are not.

In [1]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

And as we saw, we can decorate a class with it:



In [2]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


In [3]:
Person('Alex', 10).greet()

log: Person.__init__((<__main__.Person object at 0x0000023FD42FB7D0>, 'Alex', 10), {}) = None
log: Person.greet((<__main__.Person object at 0x0000023FD42FB7D0>,), {}) = Hello, my name is Alex and I am 10


'Hello, my name is Alex and I am 10'

we could achieve this functionality using the metaclass

In [10]:
class ClassLogger(type):
    def __new__(mcs, name, bases, class_dict):
        new_cls = super().__new__(mcs,name,bases,class_dict)
        for name, obj in vars(new_cls).items():
            if callable(obj):
                print('decorating:', new_cls, name)
                setattr(new_cls, name, func_logger(obj))
        return new_cls

In [11]:
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


In [12]:
p = Person('John', 78).greet()

log: Person.__init__((<__main__.Person object at 0x0000023FD46182D0>, 'John', 78), {}) = None
log: Person.greet((<__main__.Person object at 0x0000023FD46182D0>,), {}) = Hello, my name is John and I am 78


Now let's see how inheritance works with both those methods.

In [13]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


In [14]:
class Student(Person):
    def __init__(self, name, age, school):
        super().__init__(name, age)
        self.school = school

    def study(self):
        return f"{self.name} is studying.."

In [15]:
s = Student("python",30,"open")

log: Person.__init__((<__main__.Student object at 0x0000023FD5C84210>, 'python', 30), {}) = None


So first off, you can see that the print worked, but only for the __init__ in the Person class, no logs were generated for the __init__ in the Student class.

By the same token, we don't get logging on the study method

In [16]:
s.study()

'python is studying..'

So we would need to remember to decorate the Student class as well:

In [17]:
@class_logger
class Student(Person):
    def __init__(self, name, age, school):
        super().__init__(name, age)
        self.school = school

    def study(self):
        return f"{self.name} is studying.."

decorating: <class '__main__.Student'> __init__
decorating: <class '__main__.Student'> study


In [18]:
s = Student("python",30,"open")

log: Person.__init__((<__main__.Student object at 0x0000023FD4925090>, 'python', 30), {}) = None
log: Student.__init__((<__main__.Student object at 0x0000023FD4925090>, 'python', 30, 'open'), {}) = None


In [19]:
s.study()

log: Student.study((<__main__.Student object at 0x0000023FD4925090>,), {}) = python is studying..


'python is studying..'

So, we just have to remember to decorate every subclass as well.

But if we use a metaclass, watch what happens when inherit:

In [20]:
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


In [21]:
class Student(Person):
    def __init__(self, name, age, school):
        super().__init__(name, age)
        self.school = school

    def study(self):
        return f"{self.name} is studying.."

decorating: <class '__main__.Student'> __init__
decorating: <class '__main__.Student'> study


This works because Student inherits from Person, and since Person uses a metaclass for the creation, this follows down to the Student class as well.

One of the disadvantages of metaclasses vs class decorators is that only a "single" metaclass can be used. (Actually it's a bit more subtle than that, we can use a different metaclass in for a subclass if the metclass is a subclass of the parent's metaclass - we'll cover this point again when we look at multiple inheritance.)

In [22]:
class Metaclass1(type):
    pass

class Metaclass2(type):
    pass

In [23]:
class Person(metaclass=Metaclass1):
    pass

In [24]:
class Student(Person, metaclass=Metaclass2):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases